In [1]:
import sglang as sgl
sgl.set_default_backend(sgl.RuntimeEndpoint("http://localhost:30000"))

In [1]:
from sine.agents.react import ReAct
from sine.actions.action_executor import ActionExecutor
from sine.models.groq import GroqMixtralWrapper, GroqLlama70BWrapper
from sine.models.chatglm import GLM4Wrapper
from sine.models.qwen import QWENWrapper
from sine.models.kimi import MoonshotWrapper
from sine.actions.google_search import GoogleSearch

In [2]:
# model = GroqMixtralWrapper()
# model = GroqLlama70BWrapper()
# model = GLM4Wrapper()
# model = QWENWrapper()
model = MoonshotWrapper()
plugin_executor = ActionExecutor(actions=[GoogleSearch()])

react_agent = ReAct(model=model, action_executor=plugin_executor)

In [7]:
react_agent.chat("Is China Shanghai raining now?")

AgentReturn(state=<AgentStatusCode.END: 0>, actions=[ActionReturn(args={'query': 'Is it raining in Shanghai, China now?', 'k': 1}, url=None, type='GoogleSearch', result=[{'type': 'text', 'content': "['58°F']"}], errmsg=None, state=<ActionStatusCode.SUCCESS: 0>, thought=' I need to use the GoogleSearch API to check the current weather in Shanghai, China.\n', valid=<ActionValidCode.OPEN: 0>), ActionReturn(args={'query': 'Is it raining in Shanghai, China now?', 'k': 1}, url=None, type='GoogleSearch', result=[{'type': 'text', 'content': "['58°F']"}], errmsg=None, state=<ActionStatusCode.SUCCESS: 0>, thought=" The observation only provides the temperature, I need to check if it's raining or not.\n", valid=<ActionValidCode.OPEN: 0>), ActionReturn(args={'query': 'current weather conditions in Shanghai China', 'k': 1}, url=None, type='GoogleSearch', result=[{'type': 'text', 'content': "['58°F']"}], errmsg=None, state=<ActionStatusCode.SUCCESS: 0>, thought=" The API still only returned the temp

# ReAct agent summary

目的是测试并跑通各个模型使用ReAct agent上的表现，实现第一版agent(从lagent copy-paste)

尝试了的模型：Mixtral-8x7B, Llama-70B, Qwen, moonshot-v1-32k, yaofu/llama-2-7b-80k，还有一个GLM-4因为系统时间的缘故并没有跑成功；Agent代码框架：基于lagent的copy-paste

用户的问题很简单：What is the weather in Shanghai now ? 工具只有一个：GoogleSearch

## TL;DR
moonshot-v1-32k是可以完整跑完ReAct过程，并给出准确答案的模型；Qwen可以完成使用第一步使用GoogleSearch工具，但是再第二轮的过程中会报错；Mixtral-8x7B和Llama70B并没有很理解或遵循ReAct的指令，它们把ReAct的prompt的所有格式都给填了，生成了非常完美的虚构的prompt，例如虚构生成了问题的答案，并直接结束了。yaofu/llama-2-7b-80k的本地模型全是乱起八糟的回答，这个模型可以放弃了。

## conclusion
1.并不是所有的好的chat模型都有按照ReAct的指令一步一步执行的能力，例如Mixtral-8x7B和Llama-70B，其实回答都很完美，但其实是模型的幻觉。我觉得这个可能需要使用工具的数据集微调模型。(TODO: 后续可以尝试一下GorillaLlama)

2.Agent非常不稳定，容易崩溃。原因在于Agent是依靠模型的输出的文本，从中解析出action和action_input，在给到对应的action函数和函数参数；这其中模型输出的文本是不稳定的，导致提前结束或者幻觉，例如Mixtral-8x7B和Llama70B的输出幻觉严重直接就出了答案

3.由于不稳定，可想而知，过长的agent步骤目前是不可取的

4.由于不稳定，会深深陷入agent prompt engineering中，(听说Devin是heavy prompt engineering)，但是SWE-Agent的思路据说是env engineering

5.由于不稳定，类似sglang的可控LLM输出，可能会对agent更友好

In [ ]:
import requests
import os
api_key = os.environ.get('SERPER_API_KEY')

def google_search(
            api_key,
            search_term: str,
            search_type = 'search',
            **kwargs):
    headers = {
        'X-API-KEY': api_key,
        'Content-Type': 'application/json',
    }
    params = {
        'q': search_term,
        **{
            key: value
            for key, value in kwargs.items() if value is not None
        },
    }
    try:
        response = requests.post(
            f'https://google.serper.dev/{search_type}',
            headers=headers,
            params=params,
            timeout=5)
    except Exception as e:
        return -1, str(e)
    return response.status_code, response.json()

google_search(api_key, "What is the weather in Berlin now?")

(200,
 {'searchParameters': {'q': 'What is the weather in Berlin now?',
   'type': 'search',
   'engine': 'google'},
  'answerBox': {'title': 'Berlin, Germany / Weather',
   'answer': '57°F',
   'source': 'More on weather.com',
   'sourceLink': 'https://www.weather.com/wx/today/?lat=52.52&lon=13.40&locale=en_US&par=google'},
  'organic': [{'title': 'Weather Forecast and Conditions for Berlin, Germany',
    'link': 'https://weather.com/weather/today/l/5ca23443513a0fdc1d37ae2ffaf5586162c6fe592a66acc9320a0d0536be1bb9',
    'snippet': 'Berlin, Germany Forecast · Morning. 52°. Chance of Rain3% · Afternoon. 57°. Chance of Rain34% · Evening. 51°. Chance of Rain67% · Overnight. 42°. Chance of Rain ...',
    'position': 1},
   {'title': '10-Day Weather Forecast for Berlin, Germany',
    'link': 'https://weather.com/weather/tenday/l/5ca23443513a0fdc1d37ae2ffaf5586162c6fe592a66acc9320a0d0536be1bb9',
    'snippet': '10 Day Weather-Berlin, Germany. As of 9:10 am CEST. Today. 58°/41°. 50%. Mon 15 | 